In [6]:
import cv2
import numpy as np
import mediapipe as mp
import math
import argparse
from typing import List, Dict, Tuple
import os
from scipy.spatial.transform import Rotation

class MediaPipeToBVH:
    def __init__(self, fps: int = 30, scale: float = 100.0):
        """
        Initialize the MediaPipe to BVH converter.
        
        Args:
            fps: Frames per second for the output BVH file
            scale: Scaling factor for the skeleton (MediaPipe outputs in meters)
        """
        self.mp_pose = mp.solutions.pose
        self.fps = fps
        self.scale = scale
        
        # Define the BVH skeleton hierarchy - standard hierarchy that works with most 3D software
        self.joint_hierarchy = {
            "Hips": ["Spine"],
            "Spine": ["Spine1"],
            "Spine1": ["Spine2"],
            "Spine2": ["Neck", "LeftShoulder", "RightShoulder"],
            "Neck": ["Head"],
            "Head": ["End_Head"],
            "LeftShoulder": ["LeftArm"],
            "LeftArm": ["LeftForeArm"],
            "LeftForeArm": ["LeftHand"],
            "LeftHand": ["End_LeftHand"],
            "RightShoulder": ["RightArm"],
            "RightArm": ["RightForeArm"],
            "RightForeArm": ["RightHand"],
            "RightHand": ["End_RightHand"],
            "Hips": ["LeftUpLeg", "RightUpLeg", "Spine"],  # Adding children to Hips again to ensure proper order
            "LeftUpLeg": ["LeftLeg"],
            "LeftLeg": ["LeftFoot"],
            "LeftFoot": ["LeftToeBase"],
            "LeftToeBase": ["End_LeftToe"],
            "RightUpLeg": ["RightLeg"],
            "RightLeg": ["RightFoot"],
            "RightFoot": ["RightToeBase"],
            "RightToeBase": ["End_RightToe"]
        }
        
        # Clean up duplicate entries in joint hierarchy
        # (we added Hips twice to control order, now remove duplicates from lists)
        for joint, children in self.joint_hierarchy.items():
            self.joint_hierarchy[joint] = list(dict.fromkeys(children))
            
        # Define the end sites (joints with no children)
        self.end_sites = {
            "End_Head": (0, 15, 0),        # 15 units up from Head
            "End_LeftHand": (0, -15, 0),   # 15 units down from LeftHand
            "End_RightHand": (0, -15, 0),  # 15 units down from RightHand
            "End_LeftToe": (0, 0, 15),     # 15 units forward from LeftToeBase
            "End_RightToe": (0, 0, 15)     # 15 units forward from RightToeBase
        }
        
        # Root joint
        self.root_joint = "Hips"
        
        # MediaPipe landmark indices for each BVH joint
        self.landmark_to_joint = {
            "Hips": {"indices": [23, 24], "weights": [0.5, 0.5]},  # Mid-point of left hip and right hip
            "Spine": {"indices": [23, 24, 11, 12], "weights": [0.25, 0.25, 0.25, 0.25]},  # Mix of hips and shoulders
            "Spine1": {"indices": [11, 12], "weights": [0.5, 0.5]},  # Mid-point of shoulders
            "Spine2": {"indices": [11, 12], "weights": [0.5, 0.5]},  # Same as Spine1 (MediaPipe doesn't have enough spine points)
            "Neck": {"indices": [11, 12], "weights": [0.5, 0.5]},  # Base of neck
            "Head": {"indices": [0], "weights": [1.0]},  # Nose
            "LeftShoulder": {"indices": [11], "weights": [1.0]},
            "LeftArm": {"indices": [13], "weights": [1.0]},
            "LeftForeArm": {"indices": [15], "weights": [1.0]},
            "LeftHand": {"indices": [15, 17, 19, 21], "weights": [0.1, 0.3, 0.3, 0.3]},  # Weighted average of wrist and fingers
            "RightShoulder": {"indices": [12], "weights": [1.0]},
            "RightArm": {"indices": [14], "weights": [1.0]},
            "RightForeArm": {"indices": [16], "weights": [1.0]},
            "RightHand": {"indices": [16, 18, 20, 22], "weights": [0.1, 0.3, 0.3, 0.3]},  # Weighted average of wrist and fingers
            "LeftUpLeg": {"indices": [23], "weights": [1.0]},
            "LeftLeg": {"indices": [25], "weights": [1.0]},
            "LeftFoot": {"indices": [27], "weights": [1.0]},
            "LeftToeBase": {"indices": [31], "weights": [1.0]},
            "RightUpLeg": {"indices": [24], "weights": [1.0]},
            "RightLeg": {"indices": [26], "weights": [1.0]},
            "RightFoot": {"indices": [28], "weights": [1.0]},
            "RightToeBase": {"indices": [32], "weights": [1.0]}
        }
        
        # MediaPipe to BVH coordinate conversion
        # MediaPipe: +Y is up, +X is right, +Z is forward (towards camera)
        # BVH: +Y is up, +X is right, +Z is forward
        # No conversion needed for this case, but we invert Z to match typical animation systems
        self.coordinate_transform = np.array([
            [1.0, 0.0, 0.0],  # X stays the same
            [0.0, 1.0, 0.0],  # Y stays the same
            [0.0, 0.0, -1.0]  # Z is inverted
        ])
        
        # Joint bone lengths for reference
        self.bone_lengths = {}
        
        # Initialize pose tracking
        self.pose = self.mp_pose.Pose(
            static_image_mode=False,
            model_complexity=2,
            smooth_landmarks=True,
            enable_segmentation=False,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )

    def process_video(self, video_path: str) -> List[Dict]:
        """
        Process video and extract pose landmarks.
        
        Args:
            video_path: Path to the video file
            
        Returns:
            List of pose landmarks for each frame
        """
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            raise ValueError(f"Could not open video file {video_path}")
        
        # Get video properties
        self.video_fps = cap.get(cv2.CAP_PROP_FPS)
        self.video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        frames = []
        frame_count = 0
        
        while True:
            success, image = cap.read()
            if not success:
                break
            
            # Convert the BGR image to RGB
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Process the image and get pose landmarks
            results = self.pose.process(image_rgb)
            
            if results.pose_world_landmarks:
                # Convert landmarks to a dictionary format
                landmarks_dict = {}
                for i, landmark in enumerate(results.pose_world_landmarks.landmark):
                    landmarks_dict[i] = {
                        'x': landmark.x,
                        'y': landmark.y,
                        'z': landmark.z,
                        'visibility': landmark.visibility
                    }
                frames.append(landmarks_dict)
                frame_count += 1
                
                # Optional: Display processed frame with landmarks
                if frame_count % 10 == 0:
                    print(f"Processed {frame_count} frames...")
            else:
                print(f"Warning: No pose detected in frame {frame_count}. Skipping.")
                # If we already have frames, duplicate the last frame
                if frames:
                    frames.append(frames[-1])
                    frame_count += 1
        
        cap.release()
        print(f"Extracted pose data from {len(frames)} frames")
        
        if not frames:
            raise ValueError("No pose landmarks detected in the video")
            
        return frames

    def calculate_joint_positions(self, frames: List[Dict]) -> List[Dict]:
        """
        Calculate BVH joint positions from MediaPipe landmarks.
        
        Args:
            frames: List of pose landmarks for each frame
            
        Returns:
            List of dictionaries containing joint positions for each frame
        """
        joint_positions = []
        
        for frame_idx, landmarks in enumerate(frames):
            frame_positions = {}
            
            # Calculate position for each joint
            for joint_name in self.joint_hierarchy.keys():
                if joint_name.startswith("End_"):
                    continue  # End sites are handled separately
                
                mapping = self.landmark_to_joint.get(joint_name)
                if not mapping:
                    continue
                
                indices = mapping["indices"]
                weights = mapping["weights"]
                
                # Weighted average of landmark positions
                x = sum(landmarks[idx]['x'] * weights[i] for i, idx in enumerate(indices))
                y = sum(landmarks[idx]['y'] * weights[i] for i, idx in enumerate(indices))
                z = sum(landmarks[idx]['z'] * weights[i] for i, idx in enumerate(indices))
                
                # Apply coordinate transformation and scaling
                position = np.array([x, y, z])
                transformed_position = position @ self.coordinate_transform.T * self.scale
                
                frame_positions[joint_name] = {
                    'x': transformed_position[0],
                    'y': transformed_position[1],
                    'z': transformed_position[2]
                }
            
            joint_positions.append(frame_positions)
        
        # Calculate bone lengths from first good T-pose frame
        self.calculate_bone_lengths(joint_positions[0])
        
        return joint_positions

    def _smooth_joint_positions(self, joint_positions: List[Dict], window_size: int = 5) -> List[Dict]:
        """
        Apply a simple moving average filter to smooth joint positions.
        
        Args:
            joint_positions: List of dictionaries containing joint positions for each frame
            window_size: Size of the smoothing window
            
        Returns:
            Smoothed joint positions
        """
        if len(joint_positions) <= 1 or window_size <= 1:
            return joint_positions
            
        smoothed = []
        
        # Initialize with first frame
        for i in range(len(joint_positions)):
            if i < window_size // 2:
                # For the first few frames, just copy
                smoothed.append(joint_positions[i].copy())
            elif i >= len(joint_positions) - window_size // 2:
                # For the last few frames, just copy
                smoothed.append(joint_positions[i].copy())
            else:
                # Apply moving average
                smooth_frame = {}
                
                for joint_name in joint_positions[0].keys():
                    # Calculate average position over window
                    x_sum = y_sum = z_sum = 0.0
                    count = 0
                    
                    for j in range(max(0, i - window_size // 2), min(len(joint_positions), i + window_size // 2 + 1)):
                        if joint_name in joint_positions[j]:
                            x_sum += joint_positions[j][joint_name]['x']
                            y_sum += joint_positions[j][joint_name]['y']
                            z_sum += joint_positions[j][joint_name]['z']
                            count += 1
                    
                    if count > 0:
                        smooth_frame[joint_name] = {
                            'x': x_sum / count,
                            'y': y_sum / count,
                            'z': z_sum / count
                        }
                    else:
                        # If no valid data in window, use original
                        smooth_frame[joint_name] = joint_positions[i][joint_name].copy()
                
                smoothed.append(smooth_frame)
        
        return smoothed
        
    def calculate_bone_lengths(self, reference_frame: Dict):
        """
        Calculate bone lengths from a reference frame.
        
        Args:
            reference_frame: Joint positions in the reference frame
        """
        for joint_name, children in self.joint_hierarchy.items():
            if joint_name.startswith("End_"):
                continue
                
            parent_pos = np.array([
                reference_frame[joint_name]['x'],
                reference_frame[joint_name]['y'],
                reference_frame[joint_name]['z']
            ])
            
            for child in children:
                if child.startswith("End_"):
                    # End sites have fixed offsets
                    offset = np.array(self.end_sites[child])
                    self.bone_lengths[child] = offset
                    continue
                
                child_pos = np.array([
                    reference_frame[child]['x'],
                    reference_frame[child]['y'],
                    reference_frame[child]['z']
                ])
                
                # Calculate vector from parent to child
                bone_vector = child_pos - parent_pos
                
                # Store bone length
                self.bone_lengths[child] = bone_vector

    def calculate_joint_matrices(self, joint_positions: List[Dict]) -> List[Dict[str, np.ndarray]]:
        """
        Calculate joint local rotation matrices for each frame.
        
        Args:
            joint_positions: List of dictionaries containing joint positions for each frame
            
        Returns:
            List of dictionaries containing joint local rotation matrices for each frame
        """
        # Smooth the joint positions to reduce jitter (optional)
        smoothed_positions = self._smooth_joint_positions(joint_positions, window_size=5)
        joint_matrices = []
        
        # Process each frame
        for frame_idx, frame_positions in enumerate(smoothed_positions):
            frame_matrices = {}
            
            # Process each joint
            for joint_name in self.joint_hierarchy.keys():
                if joint_name.startswith("End_"):
                    continue  # Skip end sites
                
                children = self.joint_hierarchy[joint_name]
                if not children:
                    # No children, use identity matrix
                    frame_matrices[joint_name] = np.eye(3)
                    continue
                
                # Get joint position
                joint_pos = np.array([
                    frame_positions[joint_name]['x'],
                    frame_positions[joint_name]['y'],
                    frame_positions[joint_name]['z']
                ])
                
                # Collect child positions (excluding end sites)
                child_positions = []
                for child in children:
                    if child.startswith("End_"):
                        continue
                    
                    child_pos = np.array([
                        frame_positions[child]['x'],
                        frame_positions[child]['y'],
                        frame_positions[child]['z']
                    ])
                    child_positions.append(child_pos)
                
                if not child_positions:
                    # Use fixed orientations for end joints
                    if joint_name == "Head":
                        # Head looks forward
                        frame_matrices[joint_name] = np.eye(3)
                    elif joint_name.endswith("Hand"):
                        # Hands point downward
                        rot = Rotation.from_euler('xyz', [0, 0, 0], degrees=True)
                        frame_matrices[joint_name] = rot.as_matrix()
                    elif joint_name.endswith("ToeBase"):
                        # Toes point forward
                        rot = Rotation.from_euler('xyz', [0, 0, 0], degrees=True)
                        frame_matrices[joint_name] = rot.as_matrix()
                    else:
                        frame_matrices[joint_name] = np.eye(3)
                    continue
                
                # Calculate local orientation based on child positions
                if len(child_positions) == 1:
                    # Single child, direct orientation
                    child_pos = child_positions[0]
                    direction = child_pos - joint_pos
                    direction = direction / np.linalg.norm(direction)
                    
                    # For most joints, we want Y to point along the bone
                    target_vec = np.array([0, 1, 0])
                    
                    # Special cases for certain joints
                    if joint_name.endswith("Shoulder"):
                        # Shoulders point outward (left: -X, right: +X)
                        if joint_name == "LeftShoulder":
                            target_vec = np.array([-1, 0, 0])
                        else:
                            target_vec = np.array([1, 0, 0])
                    elif joint_name.endswith("UpLeg"):
                        # UpLeg points downward
                        target_vec = np.array([0, -1, 0])
                    
                    # Compute rotation from target vector to direction
                    rotation = Rotation.align_vectors(np.array([direction]), np.array([target_vec]))[0]
                    frame_matrices[joint_name] = rotation.as_matrix()
                else:
                    # Multiple children, use average direction and create local coordinate system
                    directions = []
                    for child_pos in child_positions:
                        direction = child_pos - joint_pos
                        direction = direction / np.linalg.norm(direction)
                        directions.append(direction)
                    
                    # Primary axis (Y) is the average direction to children
                    y_axis = np.mean(directions, axis=0)
                    y_axis = y_axis / np.linalg.norm(y_axis)
                    
                    # For Hips and Spine, we want to create a well-defined coordinate system
                    if joint_name in ["Hips", "Spine", "Spine1", "Spine2"]:
                        # Get left/right axis from left/right upper legs or shoulders
                        if joint_name == "Hips":
                            left_pos = np.array([
                                frame_positions["LeftUpLeg"]['x'],
                                frame_positions["LeftUpLeg"]['y'],
                                frame_positions["LeftUpLeg"]['z']
                            ])
                            right_pos = np.array([
                                frame_positions["RightUpLeg"]['x'],
                                frame_positions["RightUpLeg"]['y'],
                                frame_positions["RightUpLeg"]['z']
                            ])
                        else:
                            # Spine joints
                            left_pos = np.array([
                                frame_positions["LeftShoulder"]['x'],
                                frame_positions["LeftShoulder"]['y'],
                                frame_positions["LeftShoulder"]['z']
                            ]) if "LeftShoulder" in frame_positions else None
                            
                            right_pos = np.array([
                                frame_positions["RightShoulder"]['x'],
                                frame_positions["RightShoulder"]['y'],
                                frame_positions["RightShoulder"]['z']
                            ]) if "RightShoulder" in frame_positions else None
                        
                        if left_pos is not None and right_pos is not None:
                            # X-axis points from left to right
                            x_axis = right_pos - left_pos
                            x_axis = x_axis / np.linalg.norm(x_axis)
                            
                            # Z-axis is perpendicular to X and Y
                            z_axis = np.cross(x_axis, y_axis)
                            z_axis = z_axis / np.linalg.norm(z_axis)
                            
                            # Ensure X is perpendicular to Y and Z
                            x_axis = np.cross(y_axis, z_axis)
                            x_axis = x_axis / np.linalg.norm(x_axis)
                            
                            # Create rotation matrix
                            rotation_matrix = np.column_stack((x_axis, y_axis, z_axis))
                            frame_matrices[joint_name] = rotation_matrix
                        else:
                            # Fallback to simpler method
                            frame_matrices[joint_name] = self._create_rotation_matrix(y_axis)
                    else:
                        # For other joints with multiple children
                        frame_matrices[joint_name] = self._create_rotation_matrix(y_axis)
            
            joint_matrices.append(frame_matrices)
        
        return joint_matrices

    def _create_rotation_matrix(self, primary_axis):
        """
        Create a rotation matrix given a primary axis.
        
        Args:
            primary_axis: Primary axis for the rotation
            
        Returns:
            3x3 rotation matrix
        """
        # Check for zero vector
        norm = np.linalg.norm(primary_axis)
        if norm < 1e-10:
            # Return identity matrix if the primary axis is essentially zero
            return np.eye(3)
            
        # Ensure the primary axis is normalized
        primary_axis = primary_axis / norm
        
        # Choose a reference vector that's not parallel to primary_axis
        ref_vector = np.array([1, 0, 0])
        if np.abs(np.dot(primary_axis, ref_vector)) > 0.9:
            ref_vector = np.array([0, 1, 0])
            if np.abs(np.dot(primary_axis, ref_vector)) > 0.9:
                ref_vector = np.array([0, 0, 1])
        
        # Create a coordinate system
        secondary_axis = np.cross(primary_axis, ref_vector)
        secondary_norm = np.linalg.norm(secondary_axis)
        
        # Handle the case where secondary_axis is near zero
        if secondary_norm < 1e-10:
            # Find another reference vector
            ref_vector = np.array([0, 0, 1])
            secondary_axis = np.cross(primary_axis, ref_vector)
            secondary_norm = np.linalg.norm(secondary_axis)
            
            if secondary_norm < 1e-10:
                # If still zero, return identity with primary_axis as Y
                result = np.eye(3)
                if np.abs(primary_axis[0]) > 0.9:  # Primary axis is along X
                    result = np.array([
                        [0, 1, 0],
                        [1, 0, 0],
                        [0, 0, 1]
                    ])
                elif np.abs(primary_axis[2]) > 0.9:  # Primary axis is along Z
                    result = np.array([
                        [1, 0, 0],
                        [0, 0, 1],
                        [0, 1, 0]
                    ])
                return result
        
        secondary_axis = secondary_axis / secondary_norm
        
        tertiary_axis = np.cross(primary_axis, secondary_axis)
        tertiary_norm = np.linalg.norm(tertiary_axis)
        
        # This should not happen if the above code is correct, but just in case
        if tertiary_norm < 1e-10:
            return np.eye(3)
            
        tertiary_axis = tertiary_axis / tertiary_norm
        
        # Create rotation matrix [x, y, z] column vectors
        return np.column_stack((secondary_axis, primary_axis, tertiary_axis))

    def extract_euler_angles(self, joint_matrices: List[Dict[str, np.ndarray]]) -> List[Dict[str, Tuple[float, float, float]]]:
        """
        Extract Euler angles from joint matrices.
        
        Args:
            joint_matrices: List of dictionaries containing joint rotation matrices for each frame
            
        Returns:
            List of dictionaries containing joint Euler angles for each frame
        """
        joint_rotations = []
        
        for frame_idx, frame_matrices in enumerate(joint_matrices):
            frame_rotations = {}
            
            for joint_name, matrix in frame_matrices.items():
                try:
                    # Ensure the matrix is valid (orthogonal and proper)
                    u, s, vh = np.linalg.svd(matrix)
                    
                    # Check for reflections (det should be 1, not -1)
                    if np.linalg.det(matrix) < 0:
                        u[:, -1] *= -1
                        matrix = u @ vh
                    
                    # Make sure it's orthogonal (clean up numerical errors)
                    orthogonal_matrix = u @ vh
                    
                    # Convert matrix to Euler angles (ZXY order commonly used in BVH)
                    # Make sure determinant is close to 1 (proper rotation)
                    det = np.linalg.det(orthogonal_matrix)
                    if not 0.9 <= det <= 1.1:
                        print(f"Warning: Non-orthogonal matrix for joint '{joint_name}' in frame {frame_idx}, det={det}")
                        orthogonal_matrix = np.eye(3)  # Use identity as fallback
                    
                    rot = Rotation.from_matrix(orthogonal_matrix)
                    euler = rot.as_euler('zxy', degrees=True)
                    
                    frame_rotations[joint_name] = tuple(euler)
                except ValueError as e:
                    # If there's an error, use default orientation (no rotation)
                    print(f"Warning: Error calculating rotation for joint '{joint_name}' in frame {frame_idx}: {e}")
                    frame_rotations[joint_name] = (0.0, 0.0, 0.0)
            
            joint_rotations.append(frame_rotations)
        
        return joint_rotations

    def write_bvh_file(self, output_path: str, joint_positions: List[Dict], joint_rotations: List[Dict[str, Tuple[float, float, float]]]):
        """
        Write BVH file from joint positions and rotations.
        
        Args:
            output_path: Path to save the BVH file
            joint_positions: Joint positions for each frame
            joint_rotations: Joint rotations (Euler angles) for each frame
        """
        with open(output_path, 'w') as f:
            # Write header
            f.write("HIERARCHY\n")
            
            # Write root joint (Hips)
            f.write(f"ROOT {self.root_joint}\n")
            f.write("{\n")
            
            # Write Hips offset (typically at origin)
            f.write("\tOFFSET 0.00 0.00 0.00\n")
            
            # Write Hips channels (position + rotation)
            f.write("\tCHANNELS 6 Xposition Yposition Zposition Zrotation Xrotation Yrotation\n")
            
            # Write children recursively
            self._write_joint_hierarchy(f, self.root_joint, 1)
            
            # End of hierarchy
            f.write("}\n")
            
            # Write motion data
            f.write("MOTION\n")
            f.write(f"Frames: {len(joint_positions)}\n")
            f.write(f"Frame Time: {1.0/self.fps:.6f}\n")
            
            # Write frame data
            for frame_idx in range(len(joint_positions)):
                frame_data = []
                
                # Add root position (Hips)
                hips_pos = joint_positions[frame_idx][self.root_joint]
                frame_data.extend([hips_pos['x'], hips_pos['y'], hips_pos['z']])
                
                # Add joint rotations in hierarchy order
                self._add_joint_rotations_recursive(frame_data, joint_rotations[frame_idx], self.root_joint)
                
                # Write frame data line
                f.write(" ".join(f"{val:.6f}" for val in frame_data) + "\n")
            
            print(f"BVH file written to {output_path}")

    def _write_joint_hierarchy(self, file, joint_name: str, indent_level: int):
        """
        Recursively write joint hierarchy to BVH file.
        
        Args:
            file: File object to write to
            joint_name: Current joint name
            indent_level: Current indentation level
        """
        indent = "\t" * indent_level
        
        children = self.joint_hierarchy[joint_name]
        
        # Process each child
        for child in children:
            if child.startswith("End_"):
                # Write end site
                file.write(f"{indent}End Site\n")
                file.write(f"{indent}{{\n")
                
                # Use the predefined end site offset
                end_offset = self.end_sites[child]
                file.write(f"{indent}\tOFFSET {end_offset[0]:.6f} {end_offset[1]:.6f} {end_offset[2]:.6f}\n")
                
                file.write(f"{indent}}}\n")
            else:
                # Regular joint
                file.write(f"{indent}JOINT {child}\n")
                file.write(f"{indent}{{\n")
                
                # Write offset from parent to child
                offset = self.bone_lengths[child]
                file.write(f"{indent}\tOFFSET {offset[0]:.6f} {offset[1]:.6f} {offset[2]:.6f}\n")
                
                # Write channels (rotation only for non-root joints)
                file.write(f"{indent}\tCHANNELS 3 Zrotation Xrotation Yrotation\n")
                
                # Recursively write child's children
                self._write_joint_hierarchy(file, child, indent_level + 1)
                
                file.write(f"{indent}}}\n")

    def _add_joint_rotations_recursive(self, frame_data: List[float], rotations: Dict[str, Tuple[float, float, float]], joint_name: str):
        """
        Recursively add joint rotations to frame data in hierarchy order.
        
        Args:
            frame_data: List to append rotation data to
            rotations: Dictionary of rotations for the current frame
            joint_name: Current joint name
        """
        # Add current joint rotation (ZXY order)
        if joint_name in rotations:
            rot = rotations[joint_name]
            frame_data.extend(rot)  # ZXY order already provided by extract_euler_angles
        
        # Add children rotations in order
        for child in self.joint_hierarchy[joint_name]:
            if not child.startswith("End_"):  # Skip end sites, they have no rotation
                self._add_joint_rotations_recursive(frame_data, rotations, child)

    def convert_video_to_bvh(self, video_path: str, output_path: str):
        """
        Convert video to BVH file.
        
        Args:
            video_path: Path to input video file
            output_path: Path to save output BVH file
        """
        print(f"Processing video: {video_path}")
        frames = self.process_video(video_path)
        
        print("Calculating joint positions...")
        joint_positions = self.calculate_joint_positions(frames)
        
        print("Calculating joint orientations...")
        joint_matrices = self.calculate_joint_matrices(joint_positions)
        
        print("Extracting Euler angles...")
        joint_rotations = self.extract_euler_angles(joint_matrices)
        
        print(f"Writing BVH file to: {output_path}")
        self.write_bvh_file(output_path, joint_positions, joint_rotations)
        
        print("Conversion complete")
        return True


# def main():
#     parser = argparse.ArgumentParser(description='Convert video to BVH using MediaPipe.')
#     parser.add_argument('--input', type=str, required=True, help='Input video file')
#     parser.add_argument('--output', type=str, help='Output BVH file')
#     parser.add_argument('--fps', type=int, default=30, help='Frames per second for BVH (default: 30)')
#     parser.add_argument('--scale', type=float, default=100.0, help='Scale factor for the skeleton (default: 100.0)')
    
#     args = parser.parse_args()
    
#     # If output path is not specified, use input filename with .bvh extension
#     if not args.output:
#         base_name = os.path.splitext(os.path.basename(args.input))[0]
#         args.output = f"{base_name}.bvh"
    
#     try:
#         converter = MediaPipeToBVH(fps=args.fps, scale=args.scale)
#         converter.convert_video_to_bvh(args.input, args.output)
#         print(f"✓ BVH file created: {args.output}")
#     except Exception as e:
#         print(f"Error: {str(e)}")


# if __name__ == "__main__":
#     main()

In [7]:
filename = "fight1"
converter = MediaPipeToBVH(fps=30, scale=100.0)
converter.convert_video_to_bvh(f"{filename}.mp4", f"{filename}.bvh")

I0000 00:00:1740602941.702104   30947 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1740602941.702911   31686 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.3), renderer: Mesa Intel(R) UHD Graphics (CML GT2)
W0000 00:00:1740602941.776764   31675 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740602941.877087   31672 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Processing video: fight1.mp4
Processed 10 frames...
Processed 20 frames...
Processed 30 frames...
Processed 40 frames...
Processed 50 frames...
Processed 60 frames...
Processed 70 frames...
Processed 80 frames...
Processed 90 frames...
Extracted pose data from 91 frames
Calculating joint positions...
Calculating joint orientations...


/tmp/ipykernel_30947/1277540530.py:390: RuntimeWarning: invalid value encountered in divide
  direction = direction / np.linalg.norm(direction)


ValueError: Found zero norm quaternions in `quat`.